### Step 1: Prepare a csv file using excel (note: formatting works better with old excel version)

In [13]:
import os
import pandas as pd

In [30]:
in_csv = r"E:\HIPP\11_Deliverables\03_Bathymetry\08_SeabedTexture_And_AncillaryLayers\SI1001_Samples_WGS84_20210222.csv"
df = pd.read_csv(in_csv)
df

,SAMPLE ID,UTC Date,UTC Time,LATITUDE,LONGITUDE,Water Depth,Method,Recovery,kg,Photo Filename,Sample Composition,Folk classification,Hue Card,Hue Value,Photo filename,Sample Held,Observers Signature,Anoxic sediments,Comments,Sample,Data url,E_UTM56S,N_UTM56S
0,77,4/01/2021,00:52,-32.907482,151.863574,30.0,NaN,90,-,20210104_077,c.S.bk.Sh,Coarse sediment,10YR,10YR 6/8 (brownish yellow),20210104_077_2,N,BT,NaN,NaN,NaN,11_Deliverables\07_SeabedType\03_GrabSample_Me...,NaN,NaN
1,78,4/01/2021,01:27,-32.907612,151.881709,31.0,NaN,90,NaN,20210104_078,c.S.bk.Sh,Coarse sediment,10YR,10YR 6/8 (brownish yellow),20210104_078_2,N,BT,NaN,NaN,NaN,11_Deliverables\07_SeabedType\03_GrabSample_Me...,NaN,NaN
2,65,4/01/2021,01:51,-32.892214,151.882094,29.0,NaN,90,NaN,20210104_065,c.S.bk.Sh,Coarse sediment,10YR,10YR 6/8 (brownish yellow),20210104_065_2,N,BT,NaN,NaN,NaN,11_Deliverables\07_SeabedType\03_GrabSample_Me...,NaN,NaN
3,66,4/01/2021,02:13,-32.892475,151.900388,26.0,NaN,80,NaN,20210104_066,c.S.bk.Sh.bl.M,Coarse sediment,10YR,10YR 6/8 (brownish yellow),20210104_066_2,N,BT,NaN,NaN,NaN,11_Deliverables\07_SeabedType\03_GrabSample_Me...,NaN,NaN
4,52,4/01/2021,02:24,-32.877213,151.900391,30.0,NaN,80,NaN,20210104_052,c.S.bk.Sh,Coarse sediment,10YR,10YR 6/8 (brownish yellow),20210104_052_2,N,BT,NaN,NaN,NaN,11_Deliverables\07_SeabedType\03_GrabSample_Me...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,155,10/01/2021,15:25,-33.029942,151.843741,88.0,NaN,90,NaN,20210110_155,so.M,Coarse sediment,10YR,10YR 4/1 (dark grey),20210110_155_2,N,NS,NaN,NaN,NaN,11_Deliverables\07_SeabedType\03_GrabSample_Me...,NaN,NaN
157,154,10/01/2021,15:47,-33.029774,151.825591,78.0,NaN,90,NaN,20210110_154,so.M,Coarse sediment,10YR,10YR 4/1 (dark grey),20210110_154_2,N,NS,NaN,NaN,NaN,11_Deliverables\07_SeabedType\03_GrabSample_Me...,NaN,NaN
158,158,10/01/2021,16:11,-33.045279,151.843580,91.0,NaN,90,NaN,20210110_158,so.M,Coarse sediment,10YR,10YR 4/1 (dark grey),20210110_158_2,N,NS,NaN,NaN,NaN,11_Deliverables\07_SeabedType\03_GrabSample_Me...,NaN,NaN
159,159,10/01/2021,16:39,-33.045444,151.861780,98.0,NaN,90,NaN,20210110_159,so.M,Coarse sediment,10YR,10YR 4/1 (dark grey),20210110_159_2,N,NS,NaN,NaN,NaN,11_Deliverables\07_SeabedType\03_GrabSample_Me...,NaN,NaN


### Step 2: using XYTableToPoint to convert csv file into ESRI shp format

In [ ]:
#csv2shp_UTM56S
arcpy.env.workspace = r"E:\HIPP\11_Deliverables\03_Bathymetry\05_SoundVelocityProfiles\SVP_Raw"
in_table = r"SI1001_Samples_WGS84_20210222.csv"
out_feature_class = "SI1001_SVP_UTM56S_R2.shp"
x_coords = "E_UTM56S"
y_coords = "N_UTM56S"
z_coords = None
sr_wgs84 = arcpy.SpatialReference(32756) #CRS: UTM56S 32756
try: arcpy.management.XYTableToPoint(in_table, out_feature_class,
                                x_coords, y_coords, z_coords,
                                sr_wgs84)
except Exception as e: print(e)

# Print the total rows
print(arcpy.GetCount_management(out_feature_class))

In [ ]:
#csv2shp_WGS84
arcpy.env.workspace = r"E:\HIPP\11_Deliverables\03_Bathymetry\05_SoundVelocityProfiles\SVP_Raw"
in_table = r"SI1001_SVP_SSDM.csv"
out_feature_class = "SI1001_SVP_WGS84.shp"
x_coords = "Lon_E"
y_coords = "Lat_S"
z_coords = None
sr_wgs84 = arcpy.SpatialReference(4326) #CRS: WGS84 4326
arcpy.management.XYTableToPoint(in_table, out_feature_class,
                                x_coords, y_coords, z_coords,
                                sr_wgs84)
# Print the total rows
print(arcpy.GetCount_management(out_feature_class))

#not good

In [ ]:
#shp2csv - export and import again if wrong coordinate found
arcpy.env.workspace = r"E:\HIPP\11_Deliverables\03_Bathymetry\05_SoundVelocityProfiles\SVP_Raw"
 
# Set local variables
inTable = "SI1001_SVP_UTM56S.dbf"
outLocation = arcpy.env.workspace
outTable = "SI1001_SVP_RAW_Edit1.csv"

# Execute TableToTable
arcpy.TableToTable_conversion(inTable, outLocation, outTable)

### Step 3: Define field name and attribute values and use CalculateField to update field values

In [29]:
import arcpy
#define field names
ssdm_dict = dict()
ssdm_dict['LAST_UPDATE']='"22/02/2021"'
ssdm_dict['LAST_UPDATE_BY']='"XL"'
ssdm_dict['SURVEY_ID']='1001'
ssdm_dict['SURVEY_ID_REF']='"BN7875559"'
ssdm_dict['SAMPLING_METHOD']='"IOGP1045 Soil/Sediment Sample"'
ssdm_dict['REPORT_URL']='"11_Deliverables\01_SurveyReport_Document"'
ssdm_dict['APPROVED_DATE']='"2/03/2021"'
ssdm_dict['SAMPLING_EQUIPMENT']='"Van Veen Grab Sampling"'
ssdm_dict['PENETRATION']='0'
ssdm_dict['RECOVERY']='100'
# ssdm_dict['SYMBOLOGY_CODE']= '"IOGP1045"'

#update fields
arcpy.env.workspace = r"E:\HIPP\11_Deliverables\SSDM\GIS\FGDB\HIPP_SI1001_UTM56S.gdb"
input_ft = r'Environmental_Samples\Environmental_Sample_Pnt'
for in_field in ssdm_dict.keys():
    new_value = ssdm_dict[in_field]
    print(in_field,new_value)
    try:
        arcpy.management.CalculateField(input_ft, in_field, new_value, "PYTHON3", '', "TEXT")
    except Exception as e:
        print(e)